In [11]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import json

In [20]:
data = pd.read_csv('data_in_mave_0.csv')

In [21]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
data['title embeddings'] = ''

a = 0
for row in data['title']:
    title_embed = model.encode(row)
    data.at[a, 'title embeddings'] = title_embed
    a += 1

In [22]:
data

,Unnamed: 0,title,diffbotUri,specs,in_mave,category,main_cat,title embeddings
0,10,Master Craft 1.6 Liter Copper Watering Can,product|4|-1893862752,"{'producttypename': 'OUTDOOR_LIVING', 'binding...",True,"['Patio, Lawn & Garden', 'Gardening & Lawn Car...",Amazon Home,"[-0.28141925, 0.2698027, 0.1100372, 0.86912847..."
1,23,The Portable Henry Rollins,product|4|-1343189524,{'paperback': '320 pages'},True,"['Books', 'Literature & Fiction', 'Genre Ficti...",Books,"[-0.45946708, 0.36189297, -0.25941432, -0.2868..."
2,41,Tamron SP 24-70mm Di VC USD Nikon Mount (Model...,product|4|564903833,"{'focus_type': 'Ultrasonic', 'minimum_aperture...",True,"['Electronics', 'eBook Readers & Accessories',...",Books,"[-0.6068507, 0.34362677, 0.1731695, -0.3007811..."
3,74,Made by Dad: 67 Blueprints for Making Cool Stuff,product|4|-1826877188,{'paperback': '336 pages'},True,"['Books', 'Biographies & Memoirs', 'Arts & Lit...",Books,"[-1.2025808, 0.14240591, 0.12302717, -0.164819..."
4,82,Kai to the Rescue!,product|4|336878652,"{'grade_level': 'Preschool - Kindergarten', 'h...",True,"['Books', 'Literature & Fiction', 'Genre Ficti...",Books,"[-0.38582692, 0.8587187, 0.15365739, 0.2423255..."
...,...,...,...,...,...,...,...,...
70,2078,"Light Accents, Table Lamp 18.5 Inches Height, ...",product|4|-1280935066,"{'shade_material': 'Fabric', 'finish': 'Bronze...",True,"['Tools & Home Improvement', 'Lighting & Ceili...",Computers,"[-0.100550376, 0.63713616, 0.069718584, 0.6708..."
71,2106,The Routes Not Taken: A Trip Through New York ...,product|4|-1612441828,{'hardcover': '336 pages'},True,"['Books', 'Literature & Fiction', 'Genre Ficti...",Books,"[0.35349807, -0.8391143, 0.31275266, 0.2153156..."
72,2116,"Pono Music Portable Music Player, Black",product|4|277356526,"{'batteries_included': 'Yes', 'batteries': '1 ...",True,"['Electronics', 'Portable Audio & Video', 'MP3...",All Electronics,"[-0.38823387, -0.20821722, 0.10322529, -0.0668..."
73,2146,Out of Stock,product|4|725874185,"{'casters_included': 'Yes', 'finish': 'Grey', ...",True,"['Clothing, Shoes & Jewelry', 'Luggage & Trave...",NaN,"[-0.24847102, 0.30055243, -0.19379012, 0.12391..."


In [12]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

data['keys'] = ''
data['values'] = ''
data['title embeddings'] = ''

i = 0
for row in data['specs']:
    row_keys = list(row.keys())
    row_values = list(row.values())
    data.at[i, 'keys'] = row_keys
    data.at[i, 'values'] = row_values
    i += 1

a = 0
for row in data['title']:
    title_embed = model.encode(row)
    data.at[a, 'title embeddings'] = title_embed
    a += 1

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [ ]:
data_basic = data_basic.drop('specs', axis=1)

In [26]:
from sklearn.cluster import DBSCAN

# Cluster title embeddings
title_embeds = []

for row in data['title embeddings']:
    title_embeds.append(row)

db = DBSCAN(eps=8, min_samples=5).fit(title_embeds)
title_labels = db.labels_


titles = data['title']
specs = data['specs']
data_new = list(zip(titles, specs,  title_labels))
sorted_titles = pd.DataFrame(data_new, columns=['titles', 'specs', 'cluster_num'])
sorted_titles = sorted_titles.sort_values(by=['cluster_num'])
sorted_titles.to_csv('clusters/sorted_titles_mave.csv')
sorted_titles


# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(title_labels)) - (1 if -1 in title_labels else 0)
n_noise_ = list(title_labels).count(-1)

print("Estimated number of title clusters: %d" % n_clusters_)
print("Estimated number of title noise points: %d" % n_noise_)

Estimated number of title clusters: 1
Estimated number of title noise points: 23


In [ ]:
titles = data['title']
keys = data['keys']
data = list(zip(titles, keys, title_labels))
sorted_titles = pd.DataFrame(data, columns=['titles', 'keys', 'cluster_num'])
sorted_titles = sorted_titles.sort_values(by=['cluster_num'])
sorted_titles.to_csv('clusters/sorted_titles_mave.csv')
sorted_titles

In [54]:
import os

#for file in os.listdir('data'):
df = pd.DataFrame()

files = os.listdir('data')
index = 0
while index < 10:
    filename = files[index]
    new_df = pd.read_json(f'data/{filename}')
    df = df._append(new_df)
    index += 1

In [55]:
df = df.drop(['tokens', 'text', 'images'], axis=1)
df

,title,diffbotUri,specs
0,Offering you a complete choice of products whi...,product|4|-1577418857,"{'size': 'All Sizes', 'color': 'Black'}"
1,Acrylic Sheet in Surat,product|4|2080697291,{'specifications': '-'}
2,"Netgear ReadyNAS 312 2-Bay, 2x2TB Desktop Drive",product|4|802308696,"{'form_factor': 'Desktop', 'total_number_of_us..."
3,Stochastic Calculus for Finance II,product|4|2144040105,{'copyright_holder': 'Springer Science+Busines...
4,Stochastic Calculus for Finance II,product|4|2144040105,{'copyright_holder': 'Springer Science+Busines...
...,...,...,...
2259,Leviton 47605-MDU MDU Compact Structured Media...,product|4|-198893358,{'color': 'White'}
2260,Osiris Men’s D3R1 Skate Shoe,product|4|-1990066179,"{'department': 'mens', 'productgroup': 'Shoes'..."
2261,Ace Wood Royal Solid Color Latex Siding & Trim...,product|4|1874967644,"{'application_method': 'Brush', 'waterproof': ..."
2262,2-Piece Babysoft Bodysuit Dress & Cardigan Set,product|4|-1770906233,"{'age_of_oldest_child': '5-6', 'number_of_kids..."


In [58]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

df['keys'] = ''
df['values'] = ''
df['title embeddings'] = ''

i = 0
for row in df['specs']:
    row_keys = list(row.keys())
    row_values = list(row.values())
    print(row)
    print(row_keys, row_values)
    df.at[i, 'keys'] = row_keys
    df.at[i, 'values'] = row_values
    i += 1

a = 0
for row in df['title']:
    title_embed = model.encode(row)
    df.at[a, 'title embeddings'] = title_embed
    a += 1

{'size': 'All Sizes', 'color': 'Black'}
['size', 'color'] ['All Sizes', 'Black']


ValueError: Must have equal len keys and value when setting with an iterable

In [ ]:
df = df.drop('specs', axis=1)

In [ ]:
from sklearn.cluster import DBSCAN

# Cluster title embeddings (combined 10 MAE data parts)
title_embeds = []

for row in df['title embeddings']:
    title_embeds.append(row)

db = DBSCAN(eps=8, min_samples=5).fit(title_embeds)
title_labels = db.labels_


titles = df['title']
specs = df['specs']
df_new = list(zip(titles, specs,  title_labels))
sorted_titles = pd.DataFrame(df, columns=['titles', 'specs', 'cluster_num'])
sorted_titles = sorted_titles.sort_values(by=['cluster_num'])
sorted_titles.to_csv('clusters/sorted_titles_mae10.csv')
sorted_titles


# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(title_labels)) - (1 if -1 in title_labels else 0)
n_noise_ = list(title_labels).count(-1)

print("Estimated number of title clusters: %d" % n_clusters_)
print("Estimated number of title noise points: %d" % n_noise_)